In [288]:
import cv2
import pytesseract
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

# Specify the path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'  # Adjust according to your installation path


In [289]:
def extract_id_card_details(name, image_path):
     # Load the image
    image = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce noise and improve edge detection
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection
    edges = cv2.Canny(gray, 100, 200)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume the largest contour is the ID card (this might need tuning based on your images)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Compute the bounding rectangle for the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Extract the region of interest (ROI) using the coordinates of the rectangle
    roi = image[y:y+h, x:x+w]
    
    # Save or display the ROI
    cv2.imwrite(name, roi)
    cv2.imshow('ID Card', roi)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [290]:
def split_image(name, image_path, x, y, width=400, height=33):
    # The user wants to split the image into multiple pieces, starting with the first five rows.
    
    # Load the image
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the image dimensions
    h, w = image.shape
    
    # Define the starting point, width, and height of the rectangle
    start_x = int(w * x)
    start_y = int(h * y)
    
    # Crop the rectangle from the image
    cropped_image = image[start_y:start_y+height, start_x:start_x+width]
    
    # Save or display the cropped image
    cv2.imshow(name, cropped_image)
    
    # Optionally, save the cropped image
    path = f"split/{name}"
    cv2.imwrite(path, cropped_image)
    
    # Display the cropped image
    # plt.imshow(cropped_image)
    # plt.axis('off')  # Turn off axis numbers and ticks
    # plt.show()
    
    return path
    

In [291]:
# Function to preprocess the image
def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    

    # Convert the image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
       
    # Check the contrast by measuring the standard deviation of pixel intensities
    contrast = np.std(image)
    
    print("Contrast: ", contrast)
    
    # if contrast > 50.0:
    #     # Scale down the intensity to reduce the contrast
    #     # Assuming the image is in the range [0, 255], reduce the higher values
    #     image = np.clip(image * 0.1, 0, 15).astype(np.uint8)
    #     # Apply blur to reduce noise and improve edge detection
    #     image = cv2.blur(image,(5,5))
    # 
    # else:
    #     # Increase the contrast by scaling up the pixel intensities
    #     image = np.clip(image * 0.1, 0, 255).astype(np.uint8)
    #     # Apply blur to reduce noise and improve edge detection
    #     image = cv2.blur(image,(1,1))
        
    # Apply a mask to exclude the photo and signature if their positions are consistent
    h, w = image.shape
    mask = np.ones_like(image) * 255

    mask[0:int(h*0.89), 0:int(w*0.31)] = 0 # hide photo
    mask[:int(h*0.21), :] = 0  # hide header
    mask[int(h*0.68):, int(w*0.335):] = 0  # Applying mask to bottom right corner # Exclude signature
    
    # Apply the mask
    image = cv2.bitwise_and(image, mask)

        
    # Apply thresholding to binarize the image using Otsu's method
    _, thresh_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Resize the image
    resized_image = cv2.resize(thresh_image, (795, 500))

    # Save the processed image temporarily
    temp_processed_path = 'temp_processed_image.png'
    cv2.imwrite(temp_processed_path, resized_image)
    
    return temp_processed_path

In [292]:
# Function to perform OCR using Tesseract
def perform_ocr(temp_processed_path, cat):
    
    # Configure Tesseract to only accept alphanumeric characters (whitelist)
    if cat == 'gender':
        custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=MF'
    elif cat == 'nid':
        custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890'
    else:
        custom_config = r'--oem 3 --psm 6'
        
    # Run Tesseract OCR on the preprocessed image
    extracted_text = pytesseract.image_to_string(Image.open(temp_processed_path), config=custom_config)

    return extracted_text

In [293]:
#image_path = 'ID Summun Roshan.jpg'
image_path = 'mom.png'

In [294]:
# Preprocess the image
preprocessed_image = preprocess_image(image_path)

surname = split_image('surname.jpg', preprocessed_image, 0.33, 0.21)
forename = split_image('forename.jpg', preprocessed_image, 0.33, 0.34)
maiden = split_image('maiden.jpg', preprocessed_image, 0.33, 0.48)
gender = split_image('gender.jpg', preprocessed_image, 0.33, 0.61, width=30)
dob = split_image('dob.jpg', preprocessed_image, 0.45, 0.61, width=200)
nid = split_image('nid.jpg', preprocessed_image, 0.02, 0.88, width=250, height=50)

# Perform OCR on the preprocessed image
print("\nExtracted details:")
print(perform_ocr(surname, cat='surname').strip())
print(perform_ocr(forename, cat='forename').strip())
print(perform_ocr(maiden, cat='maiden').strip())
print(perform_ocr(gender, cat='gender').strip())
print(perform_ocr(dob, cat='dob').strip())
print(perform_ocr(nid, cat='nid').strip())


Contrast:  46.07811987507643

Extracted details:
Summun
Vedwattee Dabee
Ramgoolam
F
14 Oct 1957
R1410570139166
